In [1]:
from qdrant_client import QdrantClient, models

In [2]:
client=QdrantClient('http://localhost:6333')

In [3]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
from fastembed import TextEmbedding
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [5]:
import json 
EMBEDDING_DIMENSIONALITY = 512
for model in TextEmbedding.list_supported_models():
    if model['dim']==EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [6]:
EMBEDDING_MODEL='jinaai/jina-embeddings-v2-small-en'

In [7]:
collection_name='zoomcamp-rag'
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [8]:
points=[]
id=0

for course in documents_raw:
    for doc in course['documents']:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=EMBEDDING_MODEL), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [9]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
 def search(query, limit=1):
     results=client.query_points(
         collection_name=collection_name,
         query=models.Document(
             text=query,
             model=EMBEDDING_MODEL
         ),
         limit=limit,
         with_payload=True
     )
     return results

In [11]:
import random

course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece, indent=2))

{
  "text": "Oserror: google::cloud::status(unavailable: retry policy exhausted getbucketmetadata: could not create a OAuth2 access token to authenticate the request. the request was not sent, as such an access token is required to complete the request successfully. learn more about google cloud authentication at https://cloud.google.com/docs/authentication. the underlying error message was: performwork() - curl error [6]=couldn't resolve host name)",
  "section": "Module 2: Workflow Orchestration",
  "question": "Mage - OSError"
}


In [12]:
result = search(course_piece['question'])

In [13]:
print(f"Question: {course_piece['question']}")
print(f"\n Top Retrieved Answer:\n {result.points[0].payload['text']}")
print(f'\n Original Answer:\n {course_piece['text']}')

Question: Mage - OSError

 Top Retrieved Answer:
 Origin of Solution (Mage Slack-Channel): https://mageai.slack.com/archives/C03HTTWFEKE/p1706543947795599
Problem: This error can often be seen after solving the error mentioned in 2.2.4. The error can be found in Mage version 0.9.61 and is a side-effect of the update of the code for data-loader blocks.
Note: Mage 0.9.62 has been released, as of Feb 5 2024. Please recheck. Solution below may be obsolete
Solution: Using a “fixed” version of the docker container
Pull updated docker image from docker-hub
mageai/mageaidocker pull:alpha
Update docker-compose.yaml
version: '3'
services:
magic:
image: mageai/mageai:alpha  <--- instead of “latest”-tag
docker-compose up
The original Error is still present, but the SQL-query will return the desired result:
--------------------------------------------------------------------------------------

 Original Answer:
 Oserror: google::cloud::status(unavailable: retry policy exhausted getbucketmetadata: c

In [14]:
search('what if I submit homework late?').points[0].payload['text']

'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]'

In [15]:
client.create_payload_index(
    collection_name=collection_name,
    field_name='course',
    field_schema='keyword'
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
 def search_in_course(query,course='mlops-zoomcamp' ,limit=1):
     results=client.query_points(
         collection_name=collection_name,
         query=models.Document(
             text=query,
             model=EMBEDDING_MODEL
         ),
         query_filter=models.Filter(
             must=[
                 models.FieldCondition(
                     key='course',
                     match=models.MatchValue(value=course)
                 )
             ]
         ),
         limit=limit,
         with_payload=True
     )
     return results

In [17]:
search_in_course('what if I submit homeworks late','machine-learning-zoomcamp').points[0].payload['text']

"Depends on whether the form will still be open. If you're lucky and it's open, you can submit your homework and it will be evaluated. if closed - it's too late.\n(Added by Rileen Sinha, based on answer by Alexey on Slack)"